<a href="https://colab.research.google.com/github/pipe-felipe/tcc_knn/blob/main/tcc_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [18]:
X,y = load_iris(return_X_y=True)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=13)

In [20]:
n_neighbors = 10

In [21]:
clf = KNeighborsClassifier(n_neighbors=n_neighbors)

In [22]:
clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [23]:
y_pred = clf.predict(X_test)

In [24]:
from sklearn.metrics  import classification_report

In [25]:
iris = load_iris()
print(classification_report(y_test, y_pred, target_names=iris.target_names))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        14
  versicolor       0.80      1.00      0.89        12
   virginica       1.00      0.84      0.91        19

    accuracy                           0.93        45
   macro avg       0.93      0.95      0.93        45
weighted avg       0.95      0.93      0.93        45

